In [94]:
# 先从 将来 for loop的一个entry开始，用 tender_id = 23 作为例子

from database_util import connect_db, get_cleaned_html

db = connect_db()

# 创建游标
cursor = db.cursor()


# 这里是for loop的一个entry，也就是起点
tender_id = 23

# 获取待分析的HTML
cleaned_html = get_cleaned_html(cursor,tender_id)

# 开始写一个ai function，用来分析HTML，根据模版，输出一个dict
# 这个function 输入是 cleaned_html，输出是一个dict

# 但是我也要给 ai agent写一个tool function，按照模版，输出dict，输入就是输出。
def get_tender_detail(tender_document_start_time, tender_document_end_time, question_deadline, answer_announcement_time, bid_submission_deadline, bid_opening_time, tenderer, tender_contact, contact_phone, tender_agency, tender_agency_contact, tender_agency_contact_phone, supervision_qualification_requirement, chief_supervisor_qualification_requirement, consortium_bidding_requirement, project_name, investment_project_code, tender_project_name, implementation_site, funding_source, tender_scope_and_scale, duration, maximum_bid_price, qualification_review_method):
    return {
        "tender_document_start_time": tender_document_start_time,
        "tender_document_end_time": tender_document_end_time,
        "question_deadline": question_deadline,
        "answer_announcement_time": answer_announcement_time,
        "bid_submission_deadline": bid_submission_deadline,
        "bid_opening_time": bid_opening_time,
        "tenderer": tenderer,
        "tender_contact": tender_contact,
        "contact_phone": contact_phone,
        "tender_agency": tender_agency,
        "tender_agency_contact": tender_agency_contact,
        "tender_agency_contact_phone": tender_agency_contact_phone,
        "supervision_qualification_requirement": supervision_qualification_requirement,
        "chief_supervisor_qualification_requirement": chief_supervisor_qualification_requirement,
        "consortium_bidding_requirement": consortium_bidding_requirement,
        "project_name": project_name,
        "investment_project_code": investment_project_code,
        "tender_project_name": tender_project_name,
        "implementation_site": implementation_site,
        "funding_source": funding_source,
        "tender_scope_and_scale": tender_scope_and_scale,
        "duration": duration,
        "maximum_bid_price": maximum_bid_price,
        "qualification_review_method": qualification_review_method
    }


# 关闭游标和数据库连接
cursor.close()
db.close()





In [100]:
from zhipuai import ZhipuAI

# 现在才开始写这个ai function，输入是cleaned_html，然后爬取后，利用上面的tool function，输出一个dict
def extract_detail_from_html(cleaned_html):

    client = ZhipuAI(api_key="8d55d03f87e0ff621db27c37325c516d.4Ck0uvE4M1xYW8HB")
    tools = [
    {
        "type": "function",
        "function": {
            "name": "get_tender_detail",
            "description": "将招标详细信息插入到数据库中",
            "parameters": {
                "type": "object",
                "properties": {
                    "tender_document_start_time": {"description": "获取招标文件开始时间", "type": "string"},
                    "tender_document_end_time": {"description": "获取招标文件截止时间", "type": "string"},
                    "question_deadline": {"description": "提疑截止时间", "type": "string"},
                    "answer_announcement_time": {"description": "答疑公告时间", "type": "string"},
                    "bid_submission_deadline": {"description": "递交投标文件截止时间", "type": "string"},
                    "bid_opening_time": {"description": "开标时间", "type": "string"},
                    "tenderer": {"description": "招标人", "type": "string"},
                    "tender_contact": {"description": "招标人联系人", "type": "string"},
                    "contact_phone": {"description": "联系电话", "type": "string"},
                    "tender_agency": {"description": "招标代理机构", "type": "string"},
                    "tender_agency_contact": {"description": "招标代理联系人", "type": "string"},
                    "tender_agency_contact_phone": {"description": "招标代理联系电话", "type": "string"},
                    "supervision_qualification_requirement": {"description": "监理资质要求", "type": "string"},
                    "chief_supervisor_qualification_requirement": {"description": "总监理工程师资格要求", "type": "string"},
                    "consortium_bidding_requirement": {"description": "联合体投标要求", "type": "string"},
                    "project_name": {"description": "项目名称", "type": "string"},
                    "investment_project_code": {"description": "投资项目代码", "type": "string"},
                    "tender_project_name": {"description": "招标项目名称", "type": "string"},
                    "implementation_site": {"description": "实施地点", "type": "string"},
                    "funding_source": {"description": "资金来源", "type": "string"},
                    "tender_scope_and_scale": {"description": "招标范围及规模", "type": "string"},
                    "duration": {"description": "工期", "type": "string"},
                    "maximum_bid_price": {"description": "最高投标限价", "type": "number"},
                    "qualification_review_method": {"description": "资格审查方式", "type":  "string"}
                    },
                "required": [
                    "tenderer",
                    "tender_contact",
                    "contact_phone",
                    "tender_agency",
                    "tender_agency_contact",
                    "tender_agency_contact_phone",
                    "supervision_qualification_requirement",
                    "chief_supervisor_qualification_requirement",
                    "consortium_bidding_requirement",
                    "project_name",
                    "investment_project_code",
                    "tender_project_name",
                    "funding_source",
                    "tender_scope_and_scale",
                    "duration",
                    "maximum_bid_price",
                    "qualification_review_method"
                    ]
                }
            }
        }
    ]

    # 清空对话
    messages = []

    messages.append({"role": "system", "content": "从下面的招标公告html中，将招标详细信息提出出来，插入数据库。"})
    messages.append({"role": "user", "content":"""
                    招标公告 HTML:
                    """})

    messages.append({"role": "user", "content": cleaned_html})

    response = client.chat.completions.create(
        model="glm-4",  # 填写需要调用的模型名称
        messages=messages,
        tools=tools,
        top_p=0.7,
        temperature=0.2,
        max_tokens=4096,
    )
    return response


In [101]:
result = extract_detail_from_html(cleaned_html)

print(result)

model='glm-4' created=1709047879 choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='17:30</td>\n<th>招标人澄清问题截止时间</th>\n<td>2024-03-01 17:30</td>\n</tr>\n<tr>\n<th>投标文件递交截止时间</th>\n<td>2024-03-15 09:30</td>\n<th>投标保证金递交截止时间</th>\n<td>2024-03-15 09:30</td>\n</tr>\n<tr>\n<th>开标时间</th>\n<td>2024-03-15 09:30</td>\n</tr>\n</table>\n</div>\n', role='assistant', tool_calls=None))] request_id='8426530499720741234' id='8426530499720741234' usage=CompletionUsage(prompt_tokens=4096, completion_tokens=125, total_tokens=4221)


In [93]:
    # messages.append({"role": "system", "content": " 模版如下："})

    # messages.append({"role": "system", "content":"""
    #                 项目基本信息：
    #                 - 项目ID (整型): 唯一标识符
    #                 - 项目名称 (字符串): 详细的项目名称
    #                 - 投资项目代码 (字符串): 项目的唯一代码
    #                 - 招标项目名称 (字符串): 招标的详细名称
    #                 - 实施地点 (字符串): 具体的项目实施地点
    #                 - 资金来源 (字符串): 项目资金的来源说明
    #                 - 招标范围及规模 (文本): 详细的范围和规模描述
    #                 - 工期 (字符串): 工程预计完成时间
    #                 - 最高投标限价 (浮点数): 投标的最高价格限制
    #                 - 资格审查方式 (字符串): 资格审查的具体方法

    #                 投标人资格要求：
    #                 - 监理资质要求 (文本): 监理相关的资质要求
    #                 - 总监理工程师资格要求 (文本): 总监理工程师的资格条件
    #                 - 联合体投标要求 (文本): 关于联合体投标的具体要求

    #                 招标人和投标人信息：
    #                 - 招标人 (字符串): 招标方的名称
    #                 - 招标人联系人 (字符串): 招标方的联系人姓名
    #                 - 联系电话 (字符串): 联系人的电话号码
    #                 - 招标代理机构 (字符串): 被委托的招标代理机构名称
    #                 - 招标代理联系人 (字符串): 招标代理的联系人姓名
    #                 - 招标代理联系电话 (字符串): 招标代理联系人的电话号码

    #                 投标时间表信息：
    #                 - 获取招标文件开始时间 (字符串): 开始时间以年月日时分表示
    #                 - 获取招标文件截止时间 (字符串): 截止时间以年月日时分表示
    #                 - 提疑截止时间 (字符串): 提出疑问的截止时间以年月日时分表示
    #                 - 答疑公告时间 (字符串): 答疑公告的发布时间以年月日时分表示
    #                 - 递交投标文件截止时间 (字符串): 提交投标文件的截止时间以年月日时分表示
    #                 - 开标时间 (字符串): 开标仪式的具体时间以年月日时分表示
    #                 """})



In [83]:
response.choices[0].message

CompletionMessage(content='09:30</td>\n</tr>\n</tbody>\n</table>\n</div>\n<div>\n<p>  </p>\n</div>\n\n                 项目基本信息：\n                - 项目ID (整型): 信息缺失\n                - 项目名称 (字符串): 孙中山纪念堂加固修缮工程\n                - 投资项目代码 (字符串): 2201-442000-04-01-707876\n                - 招标项目名称 (字符串): 孙中山纪念堂加固修缮工程监理\n                - 实施地点 (字符串): 中山市石岐街道孙文中路188号\n                - 资金来源 (字符串): 市财政资金、郭得胜基金\n                - 招标范围及规模 (文本): 孙中山纪念堂加固修缮工程项目位于中山市石岐街道孙文中路188号，总用地面积合计29082.5平方米，建筑面积为9327.35平方米。主要包括：加固纪念堂建筑结构；修复建筑外墙、屋顶及修缮室内的序幕大厅、展厅、剧场、舞台、后台等；更换供配电、给排水、空调通风、消防、建筑智能系统的老化设备；对建筑物进行整体恢复性装修。项目共14栋建（构）筑物，包括13栋修缮建筑和1栋新建消防水池。立项总投资为6597.3587万元，概算建筑安装工程费为5901.30万元，本次招标监理费总报价上限为106.61万元。\n                - 工期 (字符串): 施工阶段监理工作期限暂定390个日历天，保修阶段监理工作期限24个月。\n                - 最高投标限价 (浮点数): 1066100.00元\n                - 资格审查方式 (字符串): 资格后审\n\n                投标人资格要求：\n                - 监理资质要求 (文本): 具备有效的工程监理综合资质且文物保护工程监理乙级或以上资质，或房屋建筑工程监理乙级或以上资质且文物保护工程监理乙级或以上资质\n                - 总监理工程师资格要求 (文本): 信息缺失\n 

In [84]:
tender_list = response.choices[0].message.model_dump()['content']

print(tender_list)

09:30</td>
</tr>
</tbody>
</table>
</div>
<div>
<p>  </p>
</div>

                 项目基本信息：
                - 项目ID (整型): 信息缺失
                - 项目名称 (字符串): 孙中山纪念堂加固修缮工程
                - 投资项目代码 (字符串): 2201-442000-04-01-707876
                - 招标项目名称 (字符串): 孙中山纪念堂加固修缮工程监理
                - 实施地点 (字符串): 中山市石岐街道孙文中路188号
                - 资金来源 (字符串): 市财政资金、郭得胜基金
                - 招标范围及规模 (文本): 孙中山纪念堂加固修缮工程项目位于中山市石岐街道孙文中路188号，总用地面积合计29082.5平方米，建筑面积为9327.35平方米。主要包括：加固纪念堂建筑结构；修复建筑外墙、屋顶及修缮室内的序幕大厅、展厅、剧场、舞台、后台等；更换供配电、给排水、空调通风、消防、建筑智能系统的老化设备；对建筑物进行整体恢复性装修。项目共14栋建（构）筑物，包括13栋修缮建筑和1栋新建消防水池。立项总投资为6597.3587万元，概算建筑安装工程费为5901.30万元，本次招标监理费总报价上限为106.61万元。
                - 工期 (字符串): 施工阶段监理工作期限暂定390个日历天，保修阶段监理工作期限24个月。
                - 最高投标限价 (浮点数): 1066100.00元
                - 资格审查方式 (字符串): 资格后审

                投标人资格要求：
                - 监理资质要求 (文本): 具备有效的工程监理综合资质且文物保护工程监理乙级或以上资质，或房屋建筑工程监理乙级或以上资质且文物保护工程监理乙级或以上资质
                - 总监理工程师资格要求 (文本): 信息缺失
                - 联合体投标要求 (文本): 本项目不接受联合体投标

       

In [85]:
# pip install zhipuai 请先在终端进行安装


# 清空对话
messages = []

messages.append({"role": "system", "content": " 将下面招标公告清单信息插入到数据库中"})

messages.append({"role": "user", "content": tender_list})

response_temp = client.chat.completions.create(
    model="glm-4",  # 填写需要调用的模型名称
    messages=messages,
    tools=tools,
    top_p=0.7,
    temperature=0.2,
    max_tokens=4096,
)


In [89]:
response_temp.choices[0].message.model_dump()

{'content': None,
 'role': 'assistant',
 'tool_calls': [{'id': 'call_8426529984324635676',
   'function': {'arguments': '{"answer_announcement_time":"2024-03-01 17:30","bid_opening_time":"2024-03-18 09:30","bid_submission_deadline":"2024-03-18 09:30","chief_supervisor_qualification_requirement":"信息缺失","consortium_bidding_requirement":"本项目不接受联合体投标","contact_phone":"0760-88232389","duration":"施工阶段监理工作期限暂定390个日历天，保修阶段监理工作期限24个月","funding_source":"市财政资金、郭得胜基金","implementation_site":"中山市石岐街道孙文中路188号","investment_project_code":"2201-442000-04-01-707876","maximum_bid_price":1066100,"project_name":"孙中山纪念堂加固修缮工程","qualification_review_method":"资格后审","question_deadline":"2024-02-27 17:00","supervision_qualification_requirement":"具备有效的工程监理综合资质且文物保护工程监理乙级或以上资质，或房屋建筑工程监理乙级或以上资质且文物保护工程监理乙级或以上资质","tender_agency":"广东猎德工程咨询有限公司","tender_agency_contact":"刘灿洪、何林桂","tender_agency_contact_phone":"0760-89993221、15876098160","tender_contact":"刘敏慧","tender_document_end_time":"2024-02-26 17:30","tender_documen

In [87]:
tender_list = response_temp.choices[0].message.model_dump()['content']

print(tender_list)

None


In [ ]:
from config import TENDER_DETAIL_TABLE_NAME

def insert_detail_data(ID，data):
    """
    将招标详细信息插入到数据库中。
    参数:
    - data: tuple, 要插入的数据，预期为一个元组，包含所有必要的招标详细信息字段。
    """
    tender_id = 23

    connection = connect_db()
    if connection is not None:
        try:
            cursor = connection.cursor()
            insert_query = f"""
            INSERT INTO {TENDER_DETAIL_TABLE_NAME} (tender_id, tender_document_start_time, tender_document_end_time, question_deadline, answer_announcement_time, bid_submission_deadline, bid_opening_time, tenderer, tender_contact, contact_phone, tender_agency, tender_agency_contact, tender_agency_contact_phone, supervision_qualification_requirement, business_license_requirement, chief_supervisor_qualification_requirement, consortium_bidding_requirement, project_name, investment_project_code, tender_project_name, implementation_site, funding_source, tender_scope_and_scale, duration, maximum_bid_price, qualification_review_method)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            # 为了匹配SQL插入语句，我们需要将 tender_id 添加到 data 元组的开头
            complete_data = (tender_id,) + data
            cursor.execute(insert_query, complete_data)
            connection.commit()
        except Error as e:
            print(f"Error: {e}")
            connection.rollback()
        finally:
            cursor.close()
            connection.close()


In [ ]:
from config import TENDER_DETAIL_TABLE_NAME

def insert_detail_data(data):
    """
    将招标详细信息插入到数据库中。
    参数:
    - data: tuple, 要插入的数据，预期为一个元组，包含所有必要的招标详细信息字段。
    """
    tender_id = 23

    connection = connect_db()
    if connection is not None:
        try:
            cursor = connection.cursor()
            insert_query = f"""
            INSERT INTO {TENDER_DETAIL_TABLE_NAME} (tender_id, tender_document_start_time, tender_document_end_time, question_deadline, answer_announcement_time, bid_submission_deadline, bid_opening_time, tenderer, tender_contact, contact_phone, tender_agency, tender_agency_contact, tender_agency_contact_phone, supervision_qualification_requirement, business_license_requirement, chief_supervisor_qualification_requirement, consortium_bidding_requirement, project_name, investment_project_code, tender_project_name, implementation_site, funding_source, tender_scope_and_scale, duration, maximum_bid_price, qualification_review_method)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            # 为了匹配SQL插入语句，我们需要将 tender_id 添加到 data 元组的开头
            complete_data = (tender_id,) + data
            cursor.execute(insert_query, complete_data)
            connection.commit()
        except Error as e:
            print(f"Error: {e}")
            connection.rollback()
        finally:
            cursor.close()
            connection.close()


In [38]:
# print(response.choices[0].message)

response.choices[0].message.model_dump()

{'content': None,
 'role': 'assistant',
 'tool_calls': [{'id': 'call_8426535207005064882',
   'function': {'arguments': '{"date":"2024-01-01","departure":"北京南站","destination":"上海"}',
    'name': 'query_train_info'},
   'type': 'function'}]}

In [36]:
response.choices[0].message.model_dump()['tool_calls'][0]['function']['arguments']
# import json

# temp = response.choices[0].message.tool_calls[0].function.arguments
# print(temp)

# tempp = json.loads(temp)

# tempp["date"]

'{"date":"2024-01-01","departure":"北京南站","destination":"上海"}'